In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.optim import Adam
import torch.autograd as autograd
from torch.autograd import gradcheck
from torch.utils.data import DataLoader

import random
import math
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, log_loss
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold

In [2]:
# !pip install torchquantum

In [3]:
import torchquantum as tq
from torchquantum.measurement import expval_joint_analytical
import warnings

seed = 1001
#random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.set_default_dtype(torch.float32)
warnings.simplefilter('ignore', UserWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import torchvision
from torchvision.transforms import v2

semeion_data = torchvision.datasets.SEMEION(root='./data', download=True)
dataset_name = 'semeion'
data, label = semeion_data.data, semeion_data.labels
data = data/255*math.pi/4 # pi/2
n_qubits = 8
n_class = len(np.unique(label))

data = data.reshape(-1,data.shape[1]*data.shape[2])
data = torch.from_numpy(data)
label = torch.from_numpy(label)
n_data, n_features = data.shape

Files already downloaded and verified


In [5]:
class CoeffLayer(nn.Module):
    def __init__(self, coeff):
        super().__init__()
        self.coeff = torch.nn.Parameter(coeff)
    def forward(self, x):
        ret = x * self.coeff
        return ret

class ConstCoeffLayer(nn.Module):
    def __init__(self, coeff):
        super().__init__()
        self.coeff = coeff
    def forward(self, x):
        ret = x * self.coeff
        return ret

In [6]:
# 16x16 =. 4x8x8 = 8x8x4
class QNNsubModel(nn.Module):
    def __init__(self):
        # params is numpy array
        super().__init__()
        self.n_wires = n_qubits
        self.encoder_gates_x = ([tq.functional.rx] * self.n_wires + [tq.functional.ry] * self.n_wires)*2
        self.n_block = 3
        self.n_depth_per_block = 20
        params = np.random.rand( self.n_wires*self.n_depth_per_block*self.n_block*2)*math.pi
        self.u_layers = tq.QuantumModuleList()
        for j in range(self.n_depth_per_block*self.n_block):
            for i in range(self.n_wires):
                self.u_layers.append( tq.RX(has_params=True, trainable=True, init_params=params[i+(2*j)*self.n_wires]) )
            for i in range(self.n_wires):
                self.u_layers.append( tq.RY(has_params=True, trainable=True, init_params=params[i+(2*j+1)*self.n_wires]) )

    def forward(self, x):
        bsz, nx_features = x.shape
        qdev = tq.QuantumDevice(
            n_wires=self.n_wires, bsz = bsz, device=x.device, record_op=False
        )
        n_depth_per_block = self.n_depth_per_block
        for d in range(self.n_block-1): # (2,4)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])
            # data encoding
            for j in range(2*d,2*d+2): # (0,2) (2,4)
                for k in range(self.n_wires):
                    self.encoder_gates_x[k+j*self.n_wires](qdev, wires=k, params=x[:, (k+j*self.n_wires)])
            for i in range(self.n_wires):
                qdev.cz(wires=[i,(i+1)%self.n_wires])
        for d in range(self.n_block-1,self.n_block): # (4,5)
            for k in range(n_depth_per_block):
                for j in range(2*d*n_depth_per_block+2*k,2*d*n_depth_per_block+2*k+2):
                    for i in range(self.n_wires):
                        self.u_layers[i+j*self.n_wires](qdev, wires=i)
                if k==n_depth_per_block-1:
                    break
                for i in range(self.n_wires):
                    qdev.cz(wires=[i,(i+1)%self.n_wires])

        obs_list = [ expval_joint_analytical(qdev, "I"*i+Pauli+"I"*(self.n_wires-1-i)) for Pauli in ["X","Z"] for i in range(n_class//2)]
        ret = torch.stack(obs_list, dim=1)
        return ret


In [7]:
class QNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn0, self.qnn1, self.qnn2, self.qnn3 = QNNsubModel(), QNNsubModel(), QNNsubModel(), QNNsubModel()
        self.qnn4, self.qnn5, self.qnn6, self.qnn7 = QNNsubModel(), QNNsubModel(), QNNsubModel(), QNNsubModel()
    def forward(self, x):
        LEN = 256//8
        in_x = [x[:,i*LEN:(i+1)*LEN] for i in range(8)]
        ret0, ret1, ret2, ret3 = self.qnn0(in_x[0]), self.qnn1(in_x[1]), self.qnn2(in_x[2]), self.qnn3(in_x[3])
        ret4, ret5, ret6, ret7 = self.qnn4(in_x[4]), self.qnn5(in_x[5]), self.qnn6(in_x[6]), self.qnn7(in_x[7]) 
        ret = ret0 + ret1 + ret2 + ret3 + ret4 + ret5 + ret6 + ret7
        return ret

In [8]:
def train(data, label, model, optimizer):
    model.train(mode=True)
    optimizer.zero_grad()
    pred = model(data)
    loss = torch.nn.CrossEntropyLoss()(pred, label)
    acc = (pred.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_tr, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"train loss: {loss.item():.5f}, train acc: {acc:.3f}", end=' ')
    loss.backward()
    optimizer.step()
    return loss.item(), acc

def valid(data, label, model):
    model.train(mode=False)
    with torch.no_grad():
        pred = model(data)
        loss = torch.nn.CrossEntropyLoss()(pred, label)
    acc = (pred.argmax(axis=1) == label).sum().item() / len(label)
    # acc = accuracy_score(y_te, pred.argmax(axis=1).cpu().detach().numpy() )
    print(f"valid loss: {loss.item():.5f} valid acc: {acc:.3f}")
    return loss.item(), acc

In [9]:
all_all_tr_loss = []
all_all_valid_loss = []
all_all_tr_acc = []
all_all_valid_acc = []

max_epochs = 300
lr = 0.005
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold)
for i_fold, (tr, te) in enumerate(skf.split(data, label)):
    data_tr, data_te, label_tr, label_te = data[tr].to(device), data[te].to(device), label[tr].to(device), label[te].to(device)
    model = torch.nn.Sequential(
        QNNModel(),
        ConstCoeffLayer(5),
        nn.Softmax(dim=1)
    )
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=5e-5)
    all_tr_loss = []
    all_valid_loss = []
    all_tr_acc = []
    all_valid_acc = []
    for i_epoch in range(max_epochs):
        print(f"{i_fold}-{i_epoch}th:", end=" ")
        loss_tr, acc_tr = train(data_tr, label_tr, model, optimizer)
        loss_valid, acc_valid = valid(data_te, label_te, model)
        all_tr_loss.append(loss_tr)
        all_valid_loss.append(loss_valid)
        all_tr_acc.append(acc_tr)
        all_valid_acc.append(acc_valid)
        ###
    all_all_tr_loss.append(all_tr_loss)
    all_all_valid_loss.append(all_valid_loss)
    all_all_tr_acc.append(all_tr_acc)
    all_all_valid_acc.append(all_valid_acc)

0-0th: train loss: 2.30898, train acc: 0.108 valid loss: 2.23000 valid acc: 0.304
0-1th: train loss: 2.23035, train acc: 0.274 valid loss: 2.15195 valid acc: 0.354
0-2th: train loss: 2.15245, train acc: 0.342 valid loss: 2.09804 valid acc: 0.373
0-3th: train loss: 2.09524, train acc: 0.388 valid loss: 2.05032 valid acc: 0.436
0-4th: train loss: 2.04059, train acc: 0.481 valid loss: 1.99942 valid acc: 0.549
0-5th: train loss: 1.98124, train acc: 0.586 valid loss: 1.95282 valid acc: 0.633
0-6th: train loss: 1.92534, train acc: 0.677 valid loss: 1.91218 valid acc: 0.687
0-7th: train loss: 1.87736, train acc: 0.739 valid loss: 1.87509 valid acc: 0.749
0-8th: train loss: 1.83701, train acc: 0.779 valid loss: 1.84027 valid acc: 0.796
0-9th: train loss: 1.80124, train acc: 0.808 valid loss: 1.80956 valid acc: 0.812
0-10th: train loss: 1.76946, train acc: 0.828 valid loss: 1.78494 valid acc: 0.834
0-11th: train loss: 1.74311, train acc: 0.844 valid loss: 1.76520 valid acc: 0.840
0-12th: train 

In [10]:
train_acc = []
valid_acc = []
train_loss = []
valid_loss = []
for i in range(len(all_all_tr_acc)):
    train_acc.append(all_all_tr_acc[i][-1])
    valid_acc.append(all_all_valid_acc[i][-1])
    train_loss.append(all_all_tr_loss[i][-1])
    valid_loss.append(all_all_valid_loss[i][-1])
    print(f"train acc: {train_acc[-1]}, test acc: {valid_acc[-1]}, train loss: {train_loss[-1]}, valid loss: {valid_loss[-1]}")

print( np.mean(train_acc), np.mean(valid_acc) )
print( np.std(train_acc), np.std(valid_acc) )
print( np.mean(train_loss), np.mean(valid_loss))
print( np.std(train_loss), np.std(valid_loss))

train acc: 0.9992150706436421, test acc: 0.9592476489028213, train loss: 1.4772440195083618, valid loss: 1.5526493787765503
train acc: 0.9992150706436421, test acc: 0.9467084639498433, train loss: 1.4765560626983643, valid loss: 1.5764409303665161
train acc: 0.9992150706436421, test acc: 0.9561128526645768, train loss: 1.477005958557129, valid loss: 1.5429946184158325
train acc: 0.9992156862745099, test acc: 0.940251572327044, train loss: 1.4769947528839111, valid loss: 1.571118950843811
train acc: 0.9992156862745099, test acc: 0.940251572327044, train loss: 1.4757587909698486, valid loss: 1.5811115503311157
0.9992153168959892 0.948514422034266
3.015962991683502e-07 0.007908845203017835
1.476711916923523 1.5648630857467651
0.0005258002370942716 0.014591774647785449


In [11]:
nu0 = model[0].qnn1.n_depth_per_block
c0 = int(model[1].coeff)
prefix_name = dataset_name+"16x16_"+"8qnn"+str(nu0)+"_c"+str(c0)+"_"+str(n_qubits)+"qubits_"
if False:
    pd.DataFrame(all_all_tr_acc).to_csv(prefix_name+"_tr_acc.csv", index=False)
    pd.DataFrame(all_all_valid_acc).to_csv(prefix_name+"_valid_acc.csv", index=False)
    pd.DataFrame(all_all_tr_loss).to_csv(prefix_name+"_tr_loss.csv", index=False)
    pd.DataFrame(all_all_valid_loss).to_csv(prefix_name+"_valid_loss.csv", index=False)